# Preprocessing data

## 1. Hanlding missing values

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [3]:
movies = pd.read_csv('ml-32m/movies.csv')

In [4]:
movies.head(2)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy


In [5]:
ratings = pd.read_csv('ml-32m/ratings.csv')

In [6]:
ratings.head(2)

,userId,movieId,rating,timestamp
0,1,17,4.0,944249077
1,1,25,1.0,944250228


In [7]:
tags = pd.read_csv('ml-32m/tags.csv')

In [8]:
tags.head(2)

,userId,movieId,tag,timestamp
0,22,26479,Kevin Kline,1583038886
1,22,79592,misogyny,1581476297


In [9]:
links = pd.read_csv('ml-32m/links.csv')

In [10]:
links.head(2)

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0


In [11]:
movies.isnull().sum()

movieId    0
title      0
genres     0
dtype: int64

In [12]:
tags.isnull().sum()

userId        0
movieId       0
tag          17
timestamp     0
dtype: int64

In [13]:
# tags['tag'].fillna("", inplace=True)
tags.fillna({'tag': "Unknown"}, inplace=True)

In [14]:
tags.isnull().sum()

userId       0
movieId      0
tag          0
timestamp    0
dtype: int64

In [15]:
tags[tags['tag'] =='Unknown'].head(2)

,userId,movieId,tag,timestamp
185377,27046,33826,Unknown,1221450908
1394089,89369,281500,Unknown,1670942104


In [16]:
ratings.isnull().sum()

userId       0
movieId      0
rating       0
timestamp    0
dtype: int64

In [17]:
links.isnull().sum()

movieId      0
imdbId       0
tmdbId     124
dtype: int64

In [18]:
# Fill by 0 and convert to int
cleaned_links = links.copy()
cleaned_links['tmdbId'] = cleaned_links['tmdbId'].fillna(0).astype(int)
cleaned_links.isnull().sum()

movieId    0
imdbId     0
tmdbId     0
dtype: int64

In [19]:
cleaned_links_2  = links.copy()
missing_value_records = cleaned_links_2[cleaned_links_2.isna().any(axis=1)]
missing_value_records.head()

,movieId,imdbId,tmdbId
706,721,114103,NaN
715,730,125877,NaN
754,770,38426,NaN
775,791,113610,NaN
1080,1107,102336,NaN


In [20]:
cleaned_links_2.dropna(subset=['tmdbId'], inplace=True)
cleaned_links_2.isnull().sum()

movieId    0
imdbId     0
tmdbId     0
dtype: int64

In [21]:
missing_value_records = cleaned_links_2[cleaned_links_2.isna().any(axis=1)]
missing_value_records.head()

,movieId,imdbId,tmdbId


## 2. Removing duplicates

In [22]:
print("Movies:", movies.duplicated().sum())
print("Ratings:", ratings.duplicated().sum())
print("Tags:", tags.duplicated().sum())
print("Links:", links.duplicated().sum())

Movies: 0
Ratings: 0
Tags: 0
Links: 0


In [23]:
movies.drop_duplicates(inplace=True)
ratings.drop_duplicates(inplace=True)
tags.drop_duplicates(inplace=True)
links.drop_duplicates(inplace=True)

In [24]:
# check specific columns
ratings.duplicated(subset=['userId', 'movieId']).sum()

np.int64(0)

## 3. Transforming data

#### Extract year from movie title

In [25]:
import re

In [26]:
# example
text = "The God Father (1972)"
pattern  = r'\((\d{4})\)'
match = re.search(pattern, text)
match.group(0)

'(1972)'

In [27]:
match.group(1)

'1972'

In [28]:
# Count how many movie titles that do not contain year
movies['title'].apply(lambda x: 1 if not re.search(r"\((\d{4})\)", x) else 0).sum()

np.int64(615)

In [29]:
# Add a column "year" that is extracted from column "title", if not, assign "nan"
movies_with_year = movies.copy()
# movies['title'] = movies['title'].apply(lambda x: )

In [30]:
movies_with_year['year'] = movies['title'].apply(lambda x: re.search(r"\((\d{4})\)", x).group(1) if re.search(r"\((\d{4})\)", x) else np.nan)
movies_with_year.head()

,movieId,title,genres,year
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men (1995),Comedy|Romance,1995
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II (1995),Comedy,1995


In [31]:
movies_with_year['title'].apply(lambda x: 1 if not re.search(r"\((\d{4})\)", x) else 0).head(5)

0    0
1    0
2    0
3    0
4    0
Name: title, dtype: int64

In [32]:
# real change
movies['year'] = movies['title'].apply(lambda x: re.search(r"\((\d{4})\)", x).group(1) if re.search(r"\((\d{4})\)", x) else np.nan)
movies.head()

,movieId,title,genres,year
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men (1995),Comedy|Romance,1995
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II (1995),Comedy,1995


In [33]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,17,4.0,944249077
1,1,25,1.0,944250228
2,1,29,2.0,943230976
3,1,30,5.0,944249077
4,1,32,5.0,943228858


####  Convert timestamps to a readable datetime format.

In [34]:
new_ratings = ratings.copy()
new_ratings['timestamp'] = pd.to_datetime(new_ratings['timestamp'], unit='s')
new_ratings.head()

,userId,movieId,rating,timestamp
0,1,17,4.0,1999-12-03 19:24:37
1,1,25,1.0,1999-12-03 19:43:48
2,1,29,2.0,1999-11-22 00:36:16
3,1,30,5.0,1999-12-03 19:24:37
4,1,32,5.0,1999-11-22 00:00:58


In [35]:
# real change
ratings['timestamp'] = pd.to_datetime(ratings['timestamp'], unit='s')
tags['timestamp'] = pd.to_datetime(tags['timestamp'], unit='s')

In [36]:
ratings.head(2)

,userId,movieId,rating,timestamp
0,1,17,4.0,1999-12-03 19:24:37
1,1,25,1.0,1999-12-03 19:43:48


In [37]:
tags.head(2)

,userId,movieId,tag,timestamp
0,22,26479,Kevin Kline,2020-03-01 05:01:26
1,22,79592,misogyny,2020-02-12 02:58:17


## 4. Normalzing & Scaling features

#### Normalize ratings between 0 and 1

In [38]:
# Initialize scaler
scaler = MinMaxScaler()

In [39]:
scaler.fit_transform(ratings[['rating']])

array([[0.77777778],
       [0.11111111],
       [0.33333333],
       ...,
       [0.        ],
       [0.66666667],
       [0.88888889]], shape=(32000204, 1))

In [40]:
# Apply minmax scaling to rating column
ratings['normalized_rating'] = scaler.fit_transform(ratings[['rating']])

In [41]:
ratings.head()

,userId,movieId,rating,timestamp,normalized_rating
0,1,17,4.0,1999-12-03 19:24:37,0.777778
1,1,25,1.0,1999-12-03 19:43:48,0.111111
2,1,29,2.0,1999-11-22 00:36:16,0.333333
3,1,30,5.0,1999-12-03 19:24:37,1.000000
4,1,32,5.0,1999-11-22 00:00:58,1.000000


In [42]:
# Test standardization scaling
std_scaler = StandardScaler()
std_scaler.fit_transform(ratings[['rating']])

array([[ 0.4340041 ],
       [-2.39889401],
       [-1.45459464],
       ...,
       [-2.8710437 ],
       [-0.03814559],
       [ 0.90615379]], shape=(32000204, 1))

## 5. Merging datasets

In [43]:
# Merge movies with ratings
merged_df = pd.merge(movies, ratings, on="movieId")
merged_df.head()

,movieId,title,genres,year,userId,rating,timestamp,normalized_rating
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995,10,2.5,2007-01-20 03:53:51,0.444444
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995,11,3.0,1996-12-08 22:44:36,0.555556
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995,17,4.0,2002-07-22 02:42:31,0.777778
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995,19,3.0,2000-11-20 07:14:48,0.555556
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995,20,5.0,2019-03-21 16:03:50,1.000000


In [96]:
merged_df.shape

(32000204, 8)

In [44]:
# That's enough, it didn't work
# import dask.dataframe as dd

# # Read large datasets directly as Dask DataFrames (Avoid loading full Pandas DF)
# merged_df_dd = dd.from_pandas(merged_df, npartitions=50)  # Increase partitions
# tags_dd = dd.from_pandas(tags[['movieId', 'tag']], npartitions=50)

# # Perform the merge operation with Dask
# merged_df_dd = merged_df_dd.merge(tags_dd, on="movieId", how="left")

# # Write to disk to prevent loading everything in memory
# merged_df_dd.to_csv("merged_output_*.csv", index=False, single_file=True)


## 6.Saving the cleaned data

In [45]:
# merged_df.to_csv("cleaned_movielens.csv", index=False)

In [51]:
movies.to_csv("./ml-32m-preprocessed/movies.csv", index=False)

In [50]:
ratings.head()

,userId,movieId,rating,timestamp,normalized_rating
0,1,17,4.0,1999-12-03 19:24:37,0.777778
1,1,25,1.0,1999-12-03 19:43:48,0.111111
2,1,29,2.0,1999-11-22 00:36:16,0.333333
3,1,30,5.0,1999-12-03 19:24:37,1.000000
4,1,32,5.0,1999-11-22 00:00:58,1.000000


In [52]:
ratings.to_csv("./ml-32m-preprocessed/ratings.csv", index=False)

In [53]:
links.to_csv("./ml-32m-preprocessed/links.csv", index=False)